In [ ]:
"Packages Installation "
!pip install openpyxl
!pip install pandas fiona shapely pyproj rtree 
!pip install geopandas
!pip install contextily
!pip install folium

"Libraries Import"
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import pandas as pd

I. Creation of the dataframe 

In [ ]:

df=pd.read_excel('suissedata1.xlsx')
df=df.drop(columns=['Remarque','Statut','Localité','N° commune','Ct.','Carte','Carte.1','N° sv.85','py','N° sv.','Cc','PE','PT','N° ET','Sigle ET','N° GO','Sigle GO','Nom long','Sigle sv.','PC','PP','ST'])
df.head()


#We only use the dataframe where the transport is equal to 'Zug' (Train)

df1=df.copy()

df1=df1[df1['Moyen de transport']=='Zug']
df1.head()


df2=df1[df1['Longueur']>20]
df3=df2.drop(columns=['Longueur', 'Moyen de transport','Altitude','Commune'])
df3.head()


final_df=df1.assign(Pays="Suisse")
final_df.head()
final_df.to_csv(r'Suisse\stations.csv', index = False)


In [ ]:
stations_suisse=gpd.GeoDataFrame(final_df, geometry=gpd.points_from_xy(final_df['Coord. E'], final_df['Coord. N']))
stations_suisse.crs
stations_suisse.plot()

II.1) First try to plot a complete map 

In [ ]:
"Creation of a GeoDataFrame"

crs={'init':'epsg:4326'}
map_suisse= gpd.GeoDataFrame(final_df,crs=crs, geometry=gpd.points_from_xy(final_df['Coord. E'], final_df['Coord. N']))
map_suisse.head()
map_suisse.crs

# We try to superpose 2 maps but we don't manage to because the coordinates are different between Suisse/suissedata1.xlsx and the gdf world.

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world
ax1 = world[world['name'] == 'Switzerland'].plot(
    color='white', edgecolor='black')

fig, ax = plt.subplots(figsize=(7,7))
ax1.plot(ax=ax,facecolor='Grey', edgecolor='k',alpha=1,linewidth=1,cmap="cividis")
map_suisse.plot(ax=ax,alpha=0.2,edgecolor='black',figsize=(20,20))


II.2) Second try

In [ ]:
#We use folium to plot a map
import folium
print(folium.__version__)

kanton_map = folium.Map(location=[46.8, 8.33],
                   tiles='OpenStreetMap', zoom_start=7)
kanton_map.choropleth(geo_data='/Users/aharonelbez/Desktop/Projet python 1/Suisse/mapswtzerland.geojson')
kanton_map

In [ ]:
#We try to superpose the data
from shapely.geometry import Point, Polygon

df_map=gpd.read_file('mapswtzerland.geojson')

df_map.head()

#df_map.plot()


suisse_shapefile=gpd.read_file('/Users/aharonelbez/Desktop/Projet python 1/Suisse/gadm36_CHE_shp/gadm36_CHE_2.shp')
fig, ax = plt.subplots(figsize=(7,7))
suisse_shapefile.plot(ax=ax,facecolor='white',edgecolor='k')
map_suisse.plot(ax=ax, color='red', markersize=5)

III. An ultimate solution with a correct geojson 

In [ ]:
#New geojson.
import folium
print(folium.__version__)

lines_suisse=gpd.read_file('/Users/aharonelbez/Desktop/Projet python 1/Suisse/linie-mit-polygon.geojson')

kanton_map = folium.Map(location=[46.8, 8.33],
                   tiles='OpenStreetMap', zoom_start=7)
kanton_map.choropleth(geo_data=lines_suisse)
#kanton_map
for j in range(0,len(df3)):
       folium.Marker(location=[df3.iloc[j]['Coord. E'], df3.iloc[j]['Coord. N']],popup=df3.iloc[j]['Nom (ordre alphab.)']).add_to(kanton_map)
kanton_map